### This Notebook contain scripts which makes data access, read, write/download from AWS easier and faster using AWS APIs. These scripts have to be modified to work for other AWS S3 buckets since folder paths/file paths, naming conventions were fixed for the former client.

In [ ]:
!pip install boto3

In [ ]:
import os
import boto3
import pandas
import datetime
import boto3
import concurrent.futures
import pandas as pd
import json
#import config_file as config1
import time 

## New Changes 06/15/2021 ------


In [ ]:
timestamp_start = config1.config_pre['elsa_th']['timestamp_start']
timestamp_end = config1.config_pre['elsa_th']['timestamp_end']
anomaly_high = config1.config_pre['elsa_th']['anomaly_high']
anomaly_low = config1.config_pre['elsa_th']['anomaly_low']
result_path = config1.config_pre['elsa_th']['result_path']
bucket_name = config1.config_pre['elsa_th']['bucket']
anomalous_file_path = config1.config_pre['elsa_th']['anomalous_file_path']

In [ ]:
s3_resource = boto3.resource('s3',
                aws_access_key_id = aws_access_key,
                aws_secret_access_key = aws_secret_key
            )

In [ ]:
class S3_Utility:
  def __init__(self, aws_access_key, aws_secret_key, region='us-east-2'):
    self.aws_access_key = aws_access_key
    self.aws_secret_key = aws_secret_key
    self.s3_resource = boto3.resource('s3',
                aws_access_key_id = aws_access_key,
                aws_secret_access_key = aws_secret_key,
                region_name = region
            )
    self.bucket_list = [each.name for each in self.s3_resource.buckets.all()]
 
#  def __del__(self):
#    self.s3_resource.

  def get_buckets(self):
    return self.s3_resource.buckets.all()
  
  def s3_list_files(self, bucket_name=None, Directory='', FileExtension='', timestart='1900-01-01 00:00:00', timeend='2200-01-01 00:00:00', Path=True):
    
    if timestart == '' or timestart is None:
      timestart='1900-01-01 00:00:00'
    if timeend == '' or timeend is None:
      timeend='2200-01-01 00:00:00'

    timestart = datetime.datetime.strptime(timestart+'+00:00', '%Y-%m-%d %H:%M:%S%z')
    timeend = datetime.datetime.strptime(timeend+'+00:00', '%Y-%m-%d %H:%M:%S%z')

    if (bucket_name is not None) and (bucket_name in self.bucket_list):
      filename = []
      if FileExtension != '':
        if not(Path):
          filenames = [each.key.split('/')[-1] for each in self.s3_resource.Bucket(bucket_name).objects.filter(Prefix=Directory).all() if (each.last_modified >= timestart and each.last_modified <= timeend) and each.key.split(".")[-1] == FileExtension]
        else:
          filenames = [each.key for each in self.s3_resource.Bucket(bucket_name).objects.filter(Prefix=Directory).all() if (each.last_modified >= timestart and each.last_modified <= timeend) and each.key.split(".")[-1] == FileExtension]

      else:
        if not(Path):
          filenames = [each.key.split('/')[-1] for each in self.s3_resource.Bucket(bucket_name).objects.filter(Prefix=Directory).all() if (each.last_modified >= timestart and each.last_modified <= timeend) ]      
        else:
          filenames = [each.key for each in self.s3_resource.Bucket(bucket_name).objects.filter(Prefix=Directory).all() if (each.last_modified >= timestart and each.last_modified <= timeend) ]


      return filenames
    else:
      print("Bucket Name is not correct!")

  
  def s3_download_files(self, bucket_name=None, remote_path_list=[], localpath=None, isWriteSummary=1, folderStructure=True, concurrency=None):
    # this function downloads file in a bucket.
    # remote_path is the location of file in bucket. This must be a list
    # localpath is path of directory in current working directory where you want to store files locallay on your system

    if type(remote_path_list) == list and remote_path_list != []:
      if (bucket_name is not None) and (bucket_name in self.bucket_list):
      #if not os.path.exists(os.path.join(os.getcwd(), localpath)):
        os.makedirs(localpath, exist_ok=True)
        out = []
        CONNECTIONS = concurrency #setting for how many concurrent downloads you want, currently it is getting 200 downloads concurrently?
        TIMEOUT = 60
        bucket_obj = self.s3_resource.Bucket(bucket_name)
        #config = list(map(lambda each: (bucket_obj, each, localpath, folderStructure), remote_path_list))

        def load_url(bkt, remote_file_path, local_path, fs, timeout):
          check = [each.key for each in bkt.objects.filter(Prefix=remote_file_path).all()]
          if check != []:
            if fs:
              local_path = os.path.join(local_path, check[0])
              os.makedirs(os.path.split(local_path)[0], exist_ok=True)
            else:
              local_path = os.path.join(os.getcwd(), local_path, os.path.split(remote_file_path)[-1])
            
            if not os.path.exists(local_path):
              bkt.download_file(Key=remote_file_path, Filename=local_path)
            #return [bkt.name, remote_file_path, 1]
          else:
            print("Bucket: "+bkt.name+" does not have this file: "+remote_file_path+" !")
            #return [bkt.name, remote_file_path, 0]

        with concurrent.futures.ThreadPoolExecutor(max_workers=CONNECTIONS) as executor:
          future_to_url = (map(lambda each: executor.submit(load_url, bucket_obj, each, localpath, folderStructure, TIMEOUT), remote_path_list ))
          #time1 = time.time()
          for future in concurrent.futures.as_completed(future_to_url):
            try:
              data = future.result()
            except Exception as exc:
              data = str(type(exc))
            finally:
              out.append(data)
              #print(str(len(out)),end="\r")

            #time2 = time.time()

        #print(f'Took {time2-time1:.2f} s')
        
        if isWriteSummary:
          print(out)
          pd.DataFrame(out, columns=['BucketName', 'BucketFi33lePath', 'Status']).to_csv(os.path.join(localpath, "Download_Summary.csv"), index=False)
      else:
        print("Bucket Name is not correct!")  
    else:
      print("Remote Path arguement must be a list and cannot be empty!")
    
  def s3_bucket_summary(self, bucket_name=None):
    # this function returns all files and their paths in a bucket and writes to a csv file current working directory
    # returns None
     

    if (bucket_name is not None) and (bucket_name in self.bucket_list):
      filenames = [[bucket_name, each.key, each.key.split("/")[-1]] for each in self.s3_resource.Bucket(bucket_name).objects.filter().all()]
      pd.DataFrame(filenames, columns=['Bukcet', 'FullPaths', 'FileNames']).to_csv(os.path.join(os.getcwd(), "Bucket_Summary.csv"),index=False)
    else:
      print("Bucket Name is not correct!")
  
  def s3_folder_find(self, Directory='', isWriteSummary=1):
    # this function search in all buckets for Directory and 
    # returns [bucket_name, path of that folder in bucket] also write a CSV file on 
    # current working directory

    master = []
    if Directory != '':
      for each in self.s3_resource.buckets.all():
        filenames = [[each.name, files.key[0:files.key.find(Directory)+len(Directory)]] for files in self.s3_resource.Bucket(each.name).objects.filter(Prefix=Directory).all()]
        master = master + filenames
      if isWriteSummary:
        pd.DataFrame(master, columns=['Bucket', 'Path']).drop_duplicates().to_csv(os.path.join(os.getcwd(),'Folder_Search_results.csv'), index=False)
      
      return pd.DataFrame(master).drop_duplicates().values
    else:
      print("Argument Directory cannot be empty!")

  def anomaly_download(self, bucket_name=None, timestamp_start='1900-01-01 00:00:00', timestamp_end='2260-12-31 23:59:59', anomaly_low=-100000, anomaly_high=100000, result_path_folder=None, input_csv_path=None, folderStructure=True, concurrency=None):
    ## this function downloads anomalous files filtered from csv file list 
    ## filtered parameters are timestamp range and anomalous value range
    ## "bucket_name" is good to have otherwise script will search first 
    ## part of "input_csv_file", which is actually Floc name, in all buckets
    ## if found in more than one bucket, then takes first bucket only to download 
    ## anomalous files. 
    ## parameter: "input_csv_path" is required
    ## store downloaded files in "result_path_folder"
    if timestamp_start is None or timestamp_start == '':
      timestamp_start='1900-01-01 00:00:00'
    
    if timestamp_end is None or timestamp_end == '':
      timestamp_end='2260-12-31 23:59:59'
    
    if anomaly_low is None or anomaly_low == '':
      anomaly_low = -100000

    if anomaly_high is None or anomaly_high == '':
      anomaly_high = 1000000
    
    
    if input_csv_path is None or input_csv_path == '' or input_csv_path.split(".")[-1] != 'csv':
      print("Please provide anomalous file details in csv !!!")
      return 0

    root = input_csv_path.split("/")[-1].split("_mic")[0]
    if bucket_name is None or bucket_name is '':
      buckets = self.s3_folder_find(root, 0)
      if len(buckets) == 0:
        print("Main Folder "+root+" is not found in any bucket !!!")
        return 0
      else:
        bucket_name = buckets[0][0]

    if bucket_name not in self.bucket_list:
      print("Bucket Name is not Found! Please check spellings or capatilizations in name !!!")
      return 0
    
    print("Bucket is :"+bucket_name)
    anomaly_files = pd.read_csv(input_csv_path)
    anomaly_files['fileTimestamp'] = anomaly_files['fileTimestamp'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S') )

    timestamp_start = datetime.datetime.strptime(timestamp_start, '%Y-%m-%d %H:%M:%S')
    timestamp_end = datetime.datetime.strptime(timestamp_end, '%Y-%m-%d %H:%M:%S')
    filterFiles = anomaly_files.loc[(anomaly_files['fileTimestamp'] >= timestamp_start) & (anomaly_files['fileTimestamp'] <= timestamp_end) &
                      (anomaly_files['fileAnomalousness'] >= anomaly_low) & (anomaly_files['fileAnomalousness'] <= anomaly_high) ]['filePath'].values


    #filterFiles = ['trf3-recorder-2/2021-04-16/00/TRF3_mic08_2021-04-16_00.03.00.flac']
    root = input_csv_path.split("/")[-1].split("_mic")[0]

    filteredFiles = list(map(lambda x: root+'/'+x.replace("\\",'/'), filterFiles ))
    print("Downloading "+str(len(filteredFiles))+" Files !")
    self.s3_download_files(bucket_name=bucket_name, remote_path_list=filteredFiles, localpath=result_path_folder, isWriteSummary=0, folderStructure=folderStructure, concurrency=concurrency)


In [ ]:
test = S3_Utility(aws_access_key=aws_creds['AWS_ACCESS_KEY_ID'], aws_secret_key=aws_creds['AWS_SECRET_ACCESS_KEY'])

In [ ]:
t0= time.clock()

x = test.anomaly_download(bucket_name='audiot-disk03', timestamp_start=timestamp_start, timestamp_end=timestamp_end, anomaly_high=anomaly_high, anomaly_low=anomaly_low,
                      result_path_folder=result_path, input_csv_path='/content/TRF3_2020-12_mic09_anomalousness.csv',
                       folderStructure=True, concurrency=1000)

t1 = time.clock() - t0
print("Time elapsed: ", t1)

In [ ]:
anomaly_files = pd.read_csv('TRF3_2020-12_mic09_anomalousness.csv')

filterFiles = anomaly_files['filePath'].values
    
#filterFiles = ['trf3-recorder-2/2021-04-16/00/TRF3_mic08_2021-04-16_00.03.00.flac']
root = 'TRF3_2020-12'

filteredFiles = list(map(lambda x: root+'/'+x.replace("\\",'/'), filterFiles ))[0:1000]

In [ ]:
untrained_sessions_query = """ Select bd.\"sessionId\", bd.timestamp, bd.type, bd.heart_rate, bd.qrs_interval,
                                      bd.pr_interval, bd.cluster_group
                               From (Select untrained.\"sessionId\" 
                               
                                    From ( Select distinct \"sessionId\" 
                                            From %s
                                            Where type IN (4,5,6) 
                                            Group By \"sessionId\" 
                                            Having Count(id) > %d ) untrained left join %s trained ON 
                                                                            untrained.\"sessionId\"=trained.\"sessionId\"
                                    
                                    Where trained.\"sessionId\" IS NULL) untr JOIN %s bd ON untr.\"sessionId\"=bd.\"sessionId\" """ % (table_beat, beat_threshold, table_mdl, table_beat)

In [ ]:

from concurrent import futures


bucket_name = 'audiot-disk03'

s3_object_keys = [] # List of S3 object keys


def main1(s3=None, filteredFiles=None):
  
  result_path = 'results/elsa'
  def fetch(key):
      fullpath = result_path+'/'+key
      
      os.makedirs(os.path.split(fullpath)[0], exist_ok=True) 
      
      with open(fullpath, 'wb') as data:
          s3.download_fileobj(bucket_name, key, data)
      
      return fullpath


  def fetch_all(keys):

      with futures.ThreadPoolExecutor(max_workers=2000) as executor:
          future_to_key = {executor.submit(fetch, key): key for key in keys}

          print("All URLs submitted.")

          for future in futures.as_completed(future_to_key):

              key = future_to_key[future]
              exception = future.exception()

              if not exception:
                  yield key, future.result()
              else:
                  yield key, exception


  for key, result in fetch_all(filteredFiles):
      #print(f'key: {key}  result: {result}')
      continue

s3_1 = boto3.client('s3', 
                 aws_access_key_id = aws_creds['AWS_ACCESS_KEY_ID'],
                aws_secret_access_key = aws_creds['AWS_SECRET_ACCESS_KEY'])
s3_2 = boto3.client('s3', 
                 aws_access_key_id = aws_creds['AWS_ACCESS_KEY_ID'],
                aws_secret_access_key = aws_creds['AWS_SECRET_ACCESS_KEY'])
s3_3 = boto3.client('s3', 
                 aws_access_key_id = aws_creds['AWS_ACCESS_KEY_ID'],
                aws_secret_access_key = aws_creds['AWS_SECRET_ACCESS_KEY'])
s3_4 = boto3.client('s3', 
                 aws_access_key_id = aws_creds['AWS_ACCESS_KEY_ID'],
                aws_secret_access_key = aws_creds['AWS_SECRET_ACCESS_KEY'])

main1(s3_1, filteredFiles[0:250])
main1(s3_2, filteredFiles[250:500])
main1(s3_3, filteredFiles[500:750])
main1(s3_4, filteredFiles[750:])

In [ ]:
def chunks(filteredFiles, n):
  for i in range(0, len(filteredFiles), n):
    yield filteredFiles[i:i + n]

In [ ]:

from multiprocessing import Pool

def api_call(id):
    s33 = boto3.client('s3', 
                 aws_access_key_id = aws_creds['AWS_ACCESS_KEY_ID'],
                aws_secret_access_key = aws_creds['AWS_SECRET_ACCESS_KEY'])
    main1(s33, id)

if __name__ == '__main__':
    p = Pool(4)
    p.map(api_call, chunks(filteredFiles, 250) )

All URLs submitted.
All URLs submitted.
All URLs submitted.
All URLs submitted.


In [ ]:

import time

In [ ]:
s3_resource = s3_resource = boto3.resource('s3',
                aws_access_key_id = aws_creds['AWS_ACCESS_KEY_ID'],
                aws_secret_access_key = aws_creds['AWS_SECRET_ACCESS_KEY']
            )


In [ ]:
resp = s3_resource.Object('audiot-disk03', 'TRF3_2020-12/trf3-recorder-2/2021-01-22/00/TRF3_mic09_2021-01-22_00.00.00.flac')
str(resp.last_modified)

'2021-06-04 09:38:21+00:00'

In [ ]:
[each.key for each in s3_resource.Bucket('audiot-disk03').objects.filter(Prefix='TRF3_2021-02/trf3-recorder-2/2021-04-16/00/TRF3_mic08_2021-04-16_00.03.00.flac').all()]

['TRF3_2021-02/trf3-recorder-2/2021-04-16/00/TRF3_mic08_2021-04-16_00.03.00.flac']

In [ ]:
[each.last_modified for each in s3_resource.Bucket('audiot-disk03').objects.filter(Prefix='TRF3_2020-12/trf3-recorder-2/2021-01-22/00/TRF3_mic09_2021-01-22_00.00.00.flac').all()]

[datetime.datetime(2021, 6, 4, 9, 38, 21, tzinfo=tzlocal())]

In [ ]:
[each.key for each in s3_resource.Bucket('audiot-disk03').objects.filter(Prefix='TRF3_2020-12/trf3-recorder-2/2021-01-22/00/TRF3_mic09_2021-01-22_00.00.00.flac').all()]

['TRF3_2020-12/trf3-recorder-2/2021-01-22/00/TRF3_mic09_2021-01-22_00.00.00.flac']

TRF3_2020-12\trf3-recorder-2\2021-01-22\00\TRF3_mic09_2021-01-22_00.00.00.flac

In [ ]:
with open('AWS_S3_creds.json', 'r') as f:
  aws_creds = json.loads(f.read())

## End of Changes 06/15/2021 -----

### Lists of all S3 buckets

In [ ]:
test.s3_folder_find(Directory='trf3-recorder-2\2021-01-22\02\TRF3_mic09_2021-01-22_02.48.00.flac')

[]

In [ ]:
for each in test.get_buckets().all():
  print(each.name)

audiot-disk03
audiot-practicum-2020
audiotcostuse
cough-data
dshin-sagemaker
dva-poultry-data
guy-upload-test
hyelimyang-sagemaker
jasplund-sagemaker
outside-upload-bucket
rajeevk-newbucket1
rkavadiki3--sagemaker
rzhao-sagemaker
sagemaker-studio-225277178130-8e9y2s8g15h
sagemaker-studio-225277178130-e591ugskj6j
sagemaker-studio-225277178130-f6v3hs0ga8n
sagemaker-studio-225277178130-qx122t1x93n
sagemaker-studio-us-east-1-225277178130
sagemaker-studio-us-east-1-225277178130-ktallau-chirp
sagemaker-studio-us-east-1-tyson-night-data
sagemaker-studio-us-east-1-tyson-night-data-2
sagemaker-us-east-1-225277178130
sagemaker-us-east-2-225277178130
sganesh48-sagemaker
shangao-sagechicken
tbwrf-2021-02-video
trf0-disk2
trf0-disk3
tyson-night-data
tyson-night-data-2
vetworks-upload
wilsonvetco-upload


###List of directories/sub-directories

- Function Name: `s3_list_dirs`
- Input: `bucket_name`, `sub_dir`, `paths`
    1. `bucket_name`: e.g. 'tyson-night-data'
    2. `sub_dir`: If True return list of directories as well as all nested subdirectories
    3. `paths`: If True return list with complete paths
- Output: list of directories

In [ ]:
def s3_list_dirs(bucket_name, sub_dir, path):
  subdirs = []
  dirs = []
  paths = []
  
  if sub_dir:
    bucket_info = s3_client.list_objects(Bucket=bucket_name)
    if 'Contents' in bucket_info.keys():
      for each in bucket_info['Contents']:
        temp = each['Key'].split('/')
        if path and len(temp) > 1:
          paths.append('/'.join(each['Key'].split('/')[0:-1]))
        
        if len(temp) > 2:  
          subdirs += each['Key'].split('/')[1:-1]
    else:
      print("No Sub-Directories Found in bucket: "+bucket_name)
      
  bucket_info = s3_client.list_objects(Bucket=bucket_name, Delimiter='/')
  print(bucket_info)
  if 'CommonPrefixes' in bucket_info.keys():
    for each in bucket_info['CommonPrefixes']:
      dirs.append(each['Prefix'])
  else:
    print("No Directory found in bucket: "+bucket_name)


  if path:
    return paths
  
  return list(set(dirs+subdirs))


s3_list_dirs('audiot-disk03', False, True)

{'ResponseMetadata': {'RequestId': '2N5DR4TRZ1RP0YRH', 'HostId': '0iKSNkgkXctBoNZkbNf7lGU8JpXY3Qwd2tDhBrI7HsT9elwrgekZMI+U5PxsdPnTDknDs9Dfv8Y=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': '0iKSNkgkXctBoNZkbNf7lGU8JpXY3Qwd2tDhBrI7HsT9elwrgekZMI+U5PxsdPnTDknDs9Dfv8Y=', 'x-amz-request-id': '2N5DR4TRZ1RP0YRH', 'date': 'Mon, 14 Jun 2021 20:31:10 GMT', 'x-amz-bucket-region': 'us-east-2', 'content-type': 'application/xml', 'transfer-encoding': 'chunked', 'server': 'AmazonS3'}, 'RetryAttempts': 0}, 'IsTruncated': False, 'Marker': '', 'Name': 'audiot-disk03', 'Prefix': '', 'Delimiter': '/', 'MaxKeys': 1000, 'CommonPrefixes': [{'Prefix': '$RECYCLE.BIN/'}, {'Prefix': 'System Volume Information/'}, {'Prefix': 'TRF0_2020-12/'}, {'Prefix': 'TRF0_2021-02/'}, {'Prefix': 'TRF1_2020-04and07_raw_partial/'}, {'Prefix': 'TRF1_2020-12/'}, {'Prefix': 'TRF1_2021-02/'}, {'Prefix': 'TRF2_2020-12/'}, {'Prefix': 'TRF2_2021-02/'}, {'Prefix': 'TRF3_2020-12/'}, {'Prefix': 'TRF3_2021-02/'}], 'EncodingType':

[]

###  Files

- Function Name: `s3_list_files`
- Input: `bucket_name`, `file_extension`, `paths`
    1. `bucket_name`: 'tyson-night-data'
    2. `file_extension`: Filter files based on the given file extension, if empty then list all the files
    3. `paths`: If True return list with complete paths otherwise just the name of the files
- Output: list of files

In [ ]:
def s3_list_files(bucket_name, filextension, path):
  files = []
  paths = []

  bucket_info = s3_client.list_objects(Bucket=bucket_name)
  if 'Contents' in bucket_info.keys():
    for each in bucket_info['Contents']:
      temp = each['Key'].split('/')
      if filextension == '':
        files.append(temp[-1])
        paths.append(each['Key'])

      elif filextension != '' and filextension in temp[-1]:
        files.append(temp[-1])
        paths.append(each['Key'])
  else:
    print("No File Found in bucket: "+bucket_name)
    return []

  if path:
    return paths

  return files

s3_list_files('tyson-night-data', 'flac', True)

###  Read files

- Function Name: `s3_read_files`
- Input: `file_path`
    1. `file_path`: 
- Output: File as vector

<!-- Note:  --> We might need to discuss this once you reach this task

In [ ]:
def s3_read_files(bucket_name, file_path):
  return [x for x in s3_client.get_object(Bucket=bucket_name , Key = file_path)['Body'].read()]

s3_read_files('tyson-night-data',  'Train/2020-02-08/23/TRF0_mic11_2020-02-08_23.26.30_rzha.txt')

### Download files

- Function Name: `s3_download_files`
- Input: `remote_file_path`
    1. `remote_file_path`: 
    2. `local file path`
- Output: None

<!-- Note:  --> We might need to discuss this once you reach this task